In [ ]:
from  google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 9.5 MB/s eta 0:00:00
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.2.1
    Uninstalling sentence-transformers-3.2.1:
      Successfully uninstalled sentence-transformers-3.2.1


In [ ]:
import pandas as pd
import torch
import ast
from sentence_transformers import SentenceTransformer, CrossEncoder, util


class MariannaQAChat:

    def __init__(self, file_path, encoder_model, cross_encoder_model):
        """
        Inizializza l'istanza del sistema di QA.

        :param file_path: Percorso del file TSV contenente i dati.
        :param encoder_model: Nome del modello SentenceTransformer.
        :param cross_encoder_model: Nome del modello CrossEncoder.
        """
        self.data = pd.read_csv(file_path, delimiter='\t')
        self.encoder = SentenceTransformer(encoder_model)
        self.cross_encoder = CrossEncoder(cross_encoder_model)

    def dictionary_formatter(self, data):
        """
        Converte un dataframe in una lista di dizionari nel formato {'title': title, 'text': text}.

        :param data: DataFrame di input.
        :return: Lista di dizionari formattati.
        """
        formatted_data = []

        for title, content in zip(data['title'], data['content']):
            try:
                content_dict = ast.literal_eval(content)
                for key, value in content_dict.items():
                    if isinstance(value, str):
                        formatted_data.append({'title': f"{title} - {key}", 'text': value})
                    elif isinstance(value, dict):
                        for sub_key, sub_value in value.items():
                            formatted_data.append({'title': f"{title} - {sub_key}", 'text': sub_value})
            except (ValueError, SyntaxError) as e:
                print(f"Errore nella conversione del contenuto per il titolo '{title}': {e}")

        return formatted_data

    def search_semantic_rerank(self, query, dictionaries, top_k=3):
        """
        Cerca il passaggio più rilevante per una query usando bi-encoder e cross-encoder per il reranking.

        :param query: La query in input.
        :param dictionaries: Lista di dizionari contenenti 'title' e 'text'.
        :param top_k: Numero massimo di candidati da considerare.
        :return: Il miglior passaggio.
        """
        if not dictionaries:
            print("La lista di dizionari è vuota.")
            return None

        if not isinstance(query, str) or not query.strip():
            print("La query fornita non è valida.")
            return None

        titles = [item['title'] for item in dictionaries]

        query_embedding = self.encoder.encode(query, convert_to_tensor=True)
        corpus_embeddings = self.encoder.encode(titles, convert_to_tensor=True)

        semantic_hits = util.semantic_search(query_embedding, corpus_embeddings, top_k=top_k)
        semantic_hits = semantic_hits[0]

        cross_inp = [(query, titles[hit['corpus_id']]) for hit in semantic_hits]
        cross_scores = self.cross_encoder.predict(cross_inp)

        reranked_hits = sorted(
            [{'corpus_id': hit['corpus_id'], 'cross-score': score}
             for hit, score in zip(semantic_hits, cross_scores)],
            key=lambda x: x['cross-score'], reverse=True
        )

        best_hit = reranked_hits[0]
        best_title = titles[best_hit['corpus_id']]
        best_answer = next((item['text'] for item in dictionaries if item['title'] == best_title), None)

        print(f"Query: {query}")
        print(f"Answer: {best_answer}")
        print('--------------')

        return best_answer


In [ ]:
file_path = '/content/drive/MyDrive/testa_di_Marianna/dati_wiki_ristretti/wiki_naples.tsv'

In [ ]:
Marianna_QA = MariannaQAChat(file_path,
                               'nickprock/sentence-bert-base-italian-uncased',
                               'nickprock/cross-encoder-italian-bert-stsb')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/118 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.99k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/379 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/732k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
marianna_data = Marianna_QA.data

In [ ]:
marianna_data

,summary,title,url,content,links
0,Partenope (in greco antico: Παρθενόπη?; in lat...,Partenope (città antica),https://it.wikipedia.org/wiki/Partenope_(citt%...,"{'Origini del nome': 'Partenope, che significa...","['1949', '2011', '507 a.C.', '720 a.C.', '750 ..."
1,Partenope (in greco antico: Παρθενόπη?; in lat...,Partenope (sirena),https://it.wikipedia.org/wiki/Partenope_(sirena),"{'Il mito classico': ""Secondo Esiodo Partenope...","['Acheloo', 'Ade (divinità)', 'Antonomasia', '..."
2,I decumani di Napoli sono tre antiche strade d...,Decumani di Napoli,https://it.wikipedia.org/wiki/Decumani_di_Napoli,{'Descrizione': 'I tre decumani scorrevano par...,"['Acquedotto greco-romano', 'Agorà', 'Antica G..."


In [ ]:
dicts = Marianna_QA.dictionary_formatter(marianna_data)

In [ ]:
queries = [
    "Quali sono le origini del nome Partenope?",
    "Cosa dice Tito Livio su Partenope?",
    "Cosa dice Strabone su Partenope?",
    "raccontami qualcosa del mito classico di Partenope",
    "Mi dai una descrizione dei decumani di Napoli?"
      ]

for query in queries:
    Marianna_QA.search_semantic_rerank(query, dicts)

Query: Quali sono le origini del nome Partenope?
Answer: Partenope, che significa «quella che sembra una vergine», era una delle sirene ammaliatrici che, secondo una versione di una leggenda, si suicidò lanciandosi in mare con le sorelle (Ligea e Leucosia) per l’insensibilità di Ulisse al loro canto; il suo corpo fu trasportato dalle onde alla foce di uno degli affluenti del fiume Sebeto, dove fu chiamata Parthenope la città detta poi Neapolis (Napoli).
--------------
Query: Cosa dice Tito Livio su Partenope?
Answer: Livio ne riferisce quando parla dell'assedio posto a Neapolis, nell'ambito delle guerre sannitiche. All'inizio di tale racconto viene delineata una breve descrizione sulla nascita di Neapolis: c'erano due urbes, Palepolis la città vecchia e Neapolis quella nuova, poste una accanto all'altra, abitate dal medesimo popolo e costituenti un'unica città.
Gli avvenimenti che conducono alla dedizione romana e all'allontanamento dei Sanniti ingannati, si riferiscono soltanto ad una